In [343]:
from sklearn.utils import resample
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

In [344]:
data_path = "data/total_mouse.csv"
data = pd.read_csv(data_path)
data.head()

,StartX,StartY,TargetWidth,Velocity,DirectionAngle,Duration,RectLeft,RectTop,tMouse_X,Mouse_Y,IfClick
0,-146,-28,32,64,189.254667,937,419.470269,306.462934,433,326,1
1,-168,69,96,192,35.604840,735,537.400215,462.201069,559,506,1
2,-108,147,16,256,293.784963,1151,642.278608,230.635760,632,250,0
3,-134,-124,32,128,310.895507,1162,588.309326,106.490624,585,145,0
4,170,28,64,128,195.316441,732,686.908019,331.051956,712,352,1


In [345]:
# 坐标系变换
def transform_coordinates(df):
    df['tMouse_X'] = df['tMouse_X'] - (df['RectLeft'] + df['TargetWidth'] / 2)
    df['Mouse_Y'] = -(df['Mouse_Y'] - (df['RectTop'] + df['TargetWidth'] / 2))

    angle_rad = np.deg2rad(-df['DirectionAngle'])
    tmpX = df['tMouse_X'] * np.cos(angle_rad) - df['Mouse_Y'] * np.sin(angle_rad)
    tmpY = df['tMouse_X'] * np.sin(angle_rad) + df['Mouse_Y'] * np.cos(angle_rad)
    
    df['tMouse_X'] = tmpX
    df['Mouse_Y'] = tmpY
    
    return df

# 去离群值
def clear_iso(df):
    
    mean_y = df['Mouse_Y'].mean()
    std_y = df['Mouse_Y'].std()
    mean_x = df['tMouse_X'].mean()
    std_x = df['tMouse_X'].std()
    df = df[(df['tMouse_X'] > mean_y - 3 * std_y) & (df['tMouse_X'] < mean_y + 3 * std_y)]
    df = df[(df['Mouse_Y'] > mean_x - 3 * std_x) & (df['Mouse_Y'] < mean_x + 3 * std_x)]
    return df

# 将落点平移到第一象限
def translation(df):
    df['tMouse_X'] -= df['tMouse_X'].min()
    df['Mouse_Y'] -= df['Mouse_Y'].min()
    return df

# 将落点居中
def centering(df, target_center_X, target_center_Y):
    current_center_X = df['tMouse_X'].mean()
    current_center_Y = df['Mouse_Y'].mean()
    offset_X = target_center_X - current_center_X
    offset_Y = target_center_Y - current_center_Y
    df['tMouse_X'] += offset_X 
    df['Mouse_Y'] += offset_Y
    return df

# 获取采样后图形，目前为np.array格式
def data_augmentation(df, width, velocity, repetitions, fraction):
    df = transform_coordinates(df)
    filterdata = df[(df['TargetWidth'] == width) & (df['Velocity'] == velocity)]
    filterdata = clear_iso(filterdata)
    images = []
    for _ in range(repetitions):
        sampledata = filterdata.sample(frac=fraction, replace=False)
        sampledata = translation(sampledata)
        sampledata = centering(sampledata, 128, 128)
        image = np.zeros((256, 256), dtype=float)
        for _, row in sampledata.iterrows():
            x, y = int(row['tMouse_X']), int(row['Mouse_Y'])\
            # 暂时先不考虑点重合的问题
            image[y][x] = 1
        images.append(image)
    return images, sampledata

# 获取目标图像
def get_target(width, velocity):
    image = np.zeros((256, 256), dtype=float)
    center_y, center_x = 128, 128
    for y in range(256):
        for x in range(256):
            # 计算当前点到圆心的距离
            distance = np.sqrt((center_x - x) ** 2 + (center_y - y) ** 2)
            
            # 如果距离接近半径width（可以设定一个小的容忍范围，比如±1）
            if np.abs(distance - width) <= 1:
                image[y][x]= velocity
    return image

In [346]:
dataset, sample = data_augmentation(data, 16, 64, 300, 0.8)
condition = get_target(16, 64)

## 数据预处理完成

In [352]:
dataset_tensor = torch.tensor(dataset, dtype=torch.float32)
condition_tensor = torch.tensor(condition, dtype=torch.float32)

In [358]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset_tensor, batch_size=10, shuffle=True)
epoch = 10
for i in range(epoch):
    for j, data in enumerate(dataloader):
        print(data.shape)

torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
torch.Size([10, 256, 256])
t

In [348]:
fixed_noise = torch.randn(100, 100, 1, 1)
fixed_noise.shape

torch.Size([100, 100, 1, 1])

In [ ]:
import torch.nn as nn
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.image = nn.Sequential(
            nn.ConvTranspose2d(100,
                               out_channels=256,
                               kernel_size=4,
                               stride=1,
                               padding=0,
                               bias=False
                               ),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )
        self.label = nn.Sequential(
            nn.ConvTranspose2d(num_class,
                               out_channels=256,
                               kernel_size=4,
                               stride=1,
                               padding=0,
                               bias=False
                               ),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )
        self.main = nn.Sequential(
            nn.ConvTranspose2d(512,
                               out_channels=256,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256,
                               128,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,
                               1,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False
            ),
            nn.Tanh()                
        )
    def forward(self, noise, label):
        image = self.image(noise)
        label = self.label(label)
        x = torch.cat((image, label), dim=1)
        x = self.main(x)
        return x

class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()   
        self.image = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=64,
                      kernel_size=4,
                      stride=2,
                      padding=1,
                      bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.label = nn.Sequential(
            nn.Conv2d(in_channels=num_class,
                      out_channels=64,
                      kernel_size=4,
                      stride=2,
                      padding=1,
                      bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=128,
                      out_channels=256,
                      kernel_size=4,
                      stride=2,
                      padding=1,
                      bias=False
            ),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=256,
                      out_channels=512,
                      kernel_size=4,
                      stride=2,
                      padding=1,
                      bias=False
            ),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=512,
                      out_channels=1,
                      kernel_size=4,
                      stride=1,
                      padding=0,
                      bias=False
            ),
            nn.Sigmoid()

        )
    
    def forward(self, image, label):
        image = self.image(image)
        label = self.label(label)
        x = torch.cat((image, label), dim=1)
        x = self.main(x)
        return x 

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)